In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, DatasetDict

# 1. Load dataset
dataset = load_dataset("AkashPS11/recipes_data_food.com")

# 2. Split into train and test
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
print(dataset)   # now you will see 'train' and 'test'
print(dataset['train'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

recipes.csv:   0%|          | 0.00/34.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1048543 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['RecipeId', 'Barcode', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions'],
        num_rows: 943688
    })
    test: Dataset({
        features: ['RecipeId', 'Barcode', 'Name', 'AuthorId', 'AuthorName', 'CookTime', 'PrepTime', 'TotalTime', 'DatePublished', 'Description', 'Images', 'RecipeCategory', 'Keywords', 'RecipeIngredientQuantities', 'RecipeIngredientParts', 'AggregatedRating', 'ReviewCount', 'Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent', 'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'Pr

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# LoRA config
lora_config = LoraConfig(
    r=16,           # rank of low-rank matrices
    lora_alpha=32,  # scaling
    target_modules=["c_attn"],  # where to inject LoRA
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 294,912 || all params: 82,207,488 || trainable%: 0.3587


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from datasets import load_dataset, DatasetDict

def format_example(example):
    # Return an empty string if instructions are missing
    if example["RecipeInstructions"] is None:
        return ""

    name = example.get("Name", "")
    ingredients = (
        ", ".join(example["RecipeIngredientParts"])
        if isinstance(example["RecipeIngredientParts"], list)
        else str(example["RecipeIngredientParts"] or "")
    )
    instructions = (
        " ".join(example["RecipeInstructions"])
        if isinstance(example["RecipeInstructions"], list)
        else str(example["RecipeInstructions"] or "")
    )

    return f"Title: {name}\nIngredients: {ingredients}\nInstructions: {instructions}"

# Apply formatting
dataset = dataset.map(lambda x: {"text": format_example(x)})

# Drop any empty strings left over after formatting (which were originally None)
dataset = dataset.filter(lambda x: x["text"] != "")

def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)
    encodings["labels"] = encodings["input_ids"].copy()
    return encodings

tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/943688 [00:00<?, ? examples/s]

Map:   0%|          | 0/104855 [00:00<?, ? examples/s]

Filter:   0%|          | 0/943688 [00:00<?, ? examples/s]

Filter:   0%|          | 0/104855 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nidhi03115 (nidhi03115-kmit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    num_train_epochs=2,
    learning_rate=2e-4,
    logging_steps=10,
    output_dir="./results",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shuffle().select(range(200)),  # subset for speed
    eval_dataset=tokenized_dataset["test"].shuffle().select(range(50))
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=200, training_loss=2.06144437789917, metrics={'train_runtime': 758.2706, 'train_samples_per_second': 0.528, 'train_steps_per_second': 0.264, 'total_flos': 26310868992000.0, 'train_loss': 2.06144437789917, 'epoch': 2.0})

In [ ]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-5): 6 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           

In [ ]:
import torch

text = "Title: Brownie Heart Cake\nIngredients: brown sugar, butter, vanilla, eggs, all-purpose flour, salt, white chocolate, butter, sour cream\nInstructions:"
inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True,temperature=0.9, top_p=0.9,top_k=50,repetition_penalty=1.2,no_repeat_ngram_size=3)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Title: Brownie Heart Cake
Ingredients: brown sugar, butter, vanilla, eggs, all-purpose flour, salt, white chocolate, butter, sour cream
Instructions: c("Bake until smooth.", "Remove to a lightly lined surface from the baking sheet and let cool for about 1 hour or till cake is tender. Remove with an airtight cover so no frosting falls on top.)
